In [2]:
import sys
sys.path.append('..')

import numpy as np
from pytenet.hartree_fock_mps import generate_single_state
from pytenet.operation import add_mps
from pytenet.hamiltonian_thc import eval_func, generate_thc_mpos_by_layer_qn, get_t, get_h1_spin, get_g_spin
from pytenet.global_krylov_method import generate_krylov_space_in_disk, get_W, get_S, generate_re_ortho_space_with_coeff, coeff_canonical_orthogonalization, remain_only_tridiagonal_elements
from pytenet.global_krylov_method import solve_ritz, generate_reduced_H_non_ortho, remain_only_tridiagonal_elements, coeff_gram_schmidt, generate_Hamiltonian_with_occupation_number, generate_non_ortho_krylov_space_in_disk
import numpy as np
from scipy import sparse
import copy
import h5py
from numpy.linalg import norm
#np.set_printoptions(precision=4,suppress=True)
import scipy.io
import matplotlib.pyplot as plt
import pickle
import pytenet as ptn
from scipy.linalg import cholesky, solve_triangular


Load and initialize datas: 

no is number of spatial orbitals

L is number of spinor orbitals, L = 2*no

t_spin is one-body integral in Chemist's notation (considering spins)

g_spin is two-body integral in Chemist's notation (considering spins)

X_mo and Z_mo are THC tensors, X_mo_up/down are X_mo considering spins

r_THC is THC rank

In [3]:
#load integrals
#with h5py.File("data_water/eri_water.hdf5", "r") as f:
with h5py.File("/work_fast/ge49cag/code_datas/hchain/NH_8/integral.hdf5", "r") as f:
    eri = f["eri"][()]
    hkin = f["hkin"][()]
    hnuc = f["hnuc"][()]

#print(np.linalg.norm(eri))
#print(eri.shape)

no = eri.shape[0]
MV = eri.reshape(no*no,no*no)

u = np.load("/work_fast/ge49cag/code_datas/hchain/NH_8/u.npy")
#u = np.load("/work_fast/ge49cag/pytenet_yu/water/x.npy")
X_mo = u.transpose(1,0)
g_thc, Z_mo = eval_func(u,eri,hkin,hnuc,)
h1 = hnuc+hkin
nmo = X_mo.shape[1]
L = 2*X_mo.shape[1]
g_thc = g_thc.reshape(nmo, nmo, nmo, nmo)
r_thc = X_mo.shape[0]



8
(8, 21)
(21, 21)
rl errV: 2.6368292973414723e-05
abs errV: 7.428768900793086e-05
errt: 0.00034576375364847226
errh: 4.908986218433534e-05
errht: 0.00013382891090997002


These Hamiltonian are exact molecular Hamiltonian and molecular Hamiltonian reconstructed by THC tensors. The calculation cost time, so that we store them in disk and load them when needed. For water molecule H2O in STO-6G basis, the error is small for r_THC = 28.

Actually, considering there are always 10 electrons for a water molecule, we only retain the elements which operator quantum states with 10 electrons.

In [4]:
H_correct = scipy.io.mmread('/work_fast/ge49cag/code_datas/H8_Hamiltonian.mtx').tocsr()
H_correct_8 = generate_Hamiltonian_with_occupation_number(H_correct.real, 8)
e_test, v_test = sparse.linalg.eigsh(H_correct_8, which = 'SA', k = 10)
e_ground = e_test[0]
e_1st_ex = e_test[1]

/home/ge49cag/.local/lib/python3.10/site-packages/scipy/sparse/_index.py:143: SparseEfficiencyWarning: Changing the sparsity structure of a csr_matrix is expensive. lil_matrix is more efficient.
  self._set_arrayXarray(i, j, x)


Generate THC-MPO by layers, using THC tensors. 
t_spin is used to create MPO for kinetic term.
It returns a list of H_mu_nu, each H_mu_nu is also a list, which contains four smaller MPOs with bond dims 2.
The final element of this list is MPO for kinetic term.

In [5]:
#generate thc_mpo
t = get_t(h1, eri)
H_mu_nu_list_spin_layer = generate_thc_mpos_by_layer_qn(X_mo, Z_mo, L, t)

print(type(H_mu_nu_list_spin_layer))
print(type(H_mu_nu_list_spin_layer[0]))
print(type(H_mu_nu_list_spin_layer[0][0]))
print((H_mu_nu_list_spin_layer[0][0].bond_dims))

<class 'list'>
<class 'list'>
<class 'pytenet.mpo.MPO'>
[1, 2, 2, 2, 2, 2, 2, 2, 1]


In [6]:
# h1_spin = get_h1_spin(h1)
# g_spin = get_g_spin(eri)
g_phy =  eri.transpose(0, 2, 1, 3)
#mpo_ref = ptn.hamiltonian.molecular_hamiltonian_mpo(h1_spin, g_spin_phy)
mpo_ref = ptn.hamiltonian.spin_molecular_hamiltonian_mpo(h1, g_phy)
print(mpo_ref.bond_dims)

[1, 16, 62, 100, 154, 100, 62, 16, 1]


In [7]:
#temp1 = generate_single_state(16, [1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 0, 0, 0, 0])
#initial = generate_single_state(16, [1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 0, 0, 0, 0])
HFS = generate_single_state(8, [3, 3, 3, 3, 0, 0, 0, 0])
HFS2 = generate_single_state(8, [3, 3, 3, 1, 2, 0, 0, 0])
HFS3 = generate_single_state(8, [3, 3, 3, 1, 0, 2, 0, 0])
HFS4 = generate_single_state(8, [3, 3, 3, 0, 1, 2, 0, 0])

initial = add_mps(HFS4, add_mps(HFS3, add_mps(HFS, HFS2)))
initial.orthonormalize('right')
#hartree_state = add_mps(add_mps(add_mps(temp1, temp2), temp3),temp4)
# hartree_state = add_mps(temp1, temp2)
# hartree_state.orthonormalize('left')
# hartree_state.orthonormalize('right')

2.0

In [8]:
N_Krylov = 100
psi_original = copy.deepcopy(initial)
#max_bond_Krylov = 120
max_bond_Krylov = 120
trunc_tol = 1e-11
foldername = f"/work_fast/ge49cag/code_datas/H8_Krylov"
#foldername = f"/work_fast/ge49cag/code_datas/H8_Krylov_non_ortho"
#generate_krylov_space_in_disk(N_Krylov, H_mu_nu_list_spin_layer, psi_original, max_bond_Krylov, trunc_tol, r_thc, foldername)
#generate_non_ortho_krylov_space_in_disk(N_Krylov, H_mu_nu_list_spin_layer, psi_original, max_bond_Krylov, trunc_tol, r_thc, foldername)

In [9]:
W = get_W(N_Krylov, foldername)
#coeff = coeff_gram_schmidt(N_use, foldername)

/work_fast/ge49cag/pytenet_thc_spin_cons/thc_experiments/../pytenet/global_krylov_method.py:112: ComplexWarning: Casting complex values to real discards the imaginary part
  W[i,j] = vdot(temp1, temp2)


In [10]:
size = N_Krylov

A_random = np.random.rand(N_Krylov, N_Krylov)
B = copy.deepcopy(W)  # Ensures B is symmetric positive-definite

# Ensure B is symmetric
if not np.allclose(B, B.T):
    raise ValueError("Matrix B must be symmetric.")

# Perform the Cholesky decomposition of B
try:
    L = cholesky(B, lower=True, check_finite=True)
except np.linalg.LinAlgError:
    raise ValueError("Matrix B must be positive-definite for Cholesky decomposition.")

# Instead of computing L_inv, solve the linear systems
# Compute A such that A = inv(L)
identity_matrix = np.eye(size)
A = solve_triangular(L, identity_matrix, lower=True, check_finite=True)

# Verify that A B A^T equals the identity matrix
I_computed = A @ B @ A.T
I_actual = np.identity(B.shape[0])

# Calculate the maximum off-diagonal element in I_computed
off_diagonal_elements = I_computed - np.diag(np.diag(I_computed))
max_off_diagonal = np.max(np.abs(off_diagonal_elements))

print(f"Maximum off-diagonal element in A B Aᵗ: {max_off_diagonal:.2e}")

# Check if the computed identity matrix is close to the actual identity matrix
if np.allclose(I_computed, I_actual, atol=1e-8):
    print("A B Aᵗ equals the identity matrix within tolerance.")
else:
    print("A B Aᵗ does not equal the identity matrix within tolerance.")

# Output the maximum deviation from the identity matrix
max_deviation = np.max(np.abs(I_computed - I_actual))
print(f"Maximum deviation from identity: {max_deviation:.2e}")


Maximum off-diagonal element in A B Aᵗ: 7.41e-10
A B Aᵗ equals the identity matrix within tolerance.
Maximum deviation from identity: 7.41e-10


In [11]:
#coeff = copy.deepcopy(A)
coeff = np.array(get_S(W))
W_2 = np.einsum('ca, ab, db -> cd', coeff, W, coeff)
#W_2 = np.einsum('ca, ab, db -> cd', A, W, A)
#coeff = np.array(get_S(W))
# W_2 = np.einsum('ca, ab, db -> cd', coeff, W, coeff)

# for i in range(N_Krylov):
#     for j in range(N_Krylov):
#         if i!= j:
#             #print(i, j, W_3[i, j] - W_3[j, i])
#             print(i, j, W_2[i, j])
#print(norm(W_2 - np.identity(N_Krylov)))


coeff_2_indirect = np.array(get_S(W_2))
coeff_2 = np.einsum('ki, ij -> kj', coeff_2_indirect, coeff)
W_3 = np.einsum('ca, db, ab -> cd', coeff_2, coeff_2, W)


# for i in range(N_Krylov):
#     for j in range(N_Krylov):
#         if i!= j:
#             #print(i, j, W_3[i, j] - W_3[j, i])
#             print(i, j, W_3[i, j])


coeff_3_indirect = np.array(get_S(W_3))
coeff_3 = np.einsum('ki, ij -> kj', coeff_3_indirect, coeff_2)
W_4 = np.einsum('ca, db, ab -> cd', coeff_3, coeff_3, W)

for i in range(N_Krylov):
    for j in range(N_Krylov):
        if i!= j:
            #print(i, j, W_3[i, j] - W_3[j, i])
            print(i, j, W_4[i, j])
            #assert W_4[i, j]< 1e-9
        

0 1 0.0
0 2 0.0
0 3 0.0
0 4 0.0
0 5 0.0
0 6 0.0
0 7 0.0
0 8 0.0
0 9 0.0
0 10 0.0
0 11 0.0
0 12 0.0
0 13 0.0
0 14 0.0
0 15 -4.440892098500626e-16
0 16 -1.6653345369377348e-16
0 17 7.299716386910404e-15
0 18 -6.036837696399289e-16
0 19 -1.0817735596191369e-14
0 20 -2.1843638009499955e-14
0 21 1.3012507737997225e-13
0 22 -1.3879956212159428e-13
0 23 2.6537973207840793e-13
0 24 -9.100250934757081e-14
0 25 9.095241920720198e-14
0 26 -9.679756995950584e-14
0 27 -4.163336342344337e-16
0 28 8.909539772616881e-15
0 29 -4.107825191113079e-15
0 30 1.1213252548714081e-14
0 31 1.0880185641326534e-14
0 32 1.084687895058778e-13
0 33 1.6764367671839864e-14
0 34 -4.9960036108132044e-15
0 35 -8.115730310009894e-14
0 36 -4.7128967395337895e-14
0 37 -6.938893903907228e-15
0 38 -8.415490526658687e-14
0 39 -9.992007221626409e-16
0 40 -1.5612511283791264e-15
0 41 9.853229343548264e-15
0 42 -6.480926906249351e-14
0 43 -2.7033930649622562e-14
0 44 -3.341771304121721e-14
0 45 4.6629367034256575e-15
0 46 3.77475

In [12]:
condition_number = np.linalg.cond(W)
print(f"Condition number: {condition_number:e}")


Condition number: 1.789534e+08


In [13]:
H_reduced_non_rotho = generate_reduced_H_non_ortho(N_Krylov, foldername, mpo_ref)
#coeff = coeff_canonical_orthogonalization_using_W(W)
# coeff = get_S(W)
# coeff = np.array(coeff)
#coeff = copy.deepcopy(A)

#H_reduced = np.einsum('ik, kl, jl -> ij', coeff.conj(), H_reduced_non_rotho, coeff)
H_reduced = np.einsum('ik, kl, jl -> ij', coeff_3.conj(), H_reduced_non_rotho, coeff_3)
#H_reduced = remain_only_tridiagonal_elements(H_reduced)

/work_fast/ge49cag/pytenet_thc_spin_cons/thc_experiments/../pytenet/global_krylov_method.py:213: ComplexWarning: Casting complex values to real discards the imaginary part
  H_reduced[i, j] = operator_inner_product(temp1, H_mpo, temp2)


In [14]:
e_ritz_1, v_ritz_1 = solve_ritz(foldername, H_reduced, N_Krylov, coeff, max_bond_Krylov, e_ground, mpo_ref)

(3.1042153363340885+0j)
(1.5440463485601885+0j)
(0.6290819230014755+0j)
(0.22954232987015288+0j)
(0.0027207952378276445+0j)
(0.00016241233081970563+0j)
(2.809126097247372e-05+0j)
(9.2887133824604e-06+0j)
(6.6632077349026986e-06+0j)
(6.509256358810944e-06+0j)
(6.4992550967701845e-06+0j)
(6.499056731001929e-06+0j)
(6.497037498220948e-06+0j)
(6.465575870251428e-06+0j)
(6.441446890548264e-06+0j)
(6.412531480037842e-06+0j)
(6.403806811050572e-06+0j)
(6.372801943044237e-06+0j)
(6.381641671993066e-06+0j)
(6.421684787483173e-06+0j)


In [15]:
np.linalg.eigh(H_reduced[:50, :50])

EighResult(eigenvalues=array([-9.28650196, -9.23252714, -9.16598347, -9.09972924, -8.92404696,
       -8.86381176, -8.81186883, -8.70639433, -8.62248891, -8.53053193,
       -8.41072501, -8.29181193, -8.16936057, -8.01742229, -7.89024721,
       -7.76843916, -7.6276091 , -7.47422693, -7.28400995, -7.1963251 ,
       -7.05514602, -6.97841047, -6.92467197, -6.80774451, -6.72758271,
       -6.66565831, -6.51696654, -6.37977529, -6.25737067, -6.18846535,
       -6.112763  , -6.03879964, -5.91838595, -5.80705226, -5.70073724,
       -5.60639585, -5.49980834, -5.4717945 , -5.29459172, -5.23211963,
       -5.13489527, -5.10789151, -5.04620272, -5.00117489, -4.95289007,
       -4.9124242 , -4.84588116, -4.7016994 , -4.65569461, -3.91239404]), eigenvectors=array([[ 3.86350742e-10, -4.14230520e-09,  1.94106450e-08, ...,
        -2.73118614e-03,  1.96254493e-01, -9.68202600e-01],
       [-1.57200610e-09,  1.01451009e-08, -1.40318732e-08, ...,
         8.42997601e-03, -5.67047196e-01, -2.39565084e